In [32]:
import matplotlib.pyplot as plt
import matplotlib as mpl # 한글 폰트 설정 (NanumGothic)
import pandas as pd 
import seaborn as sns
import scipy.stats as stats


mpl.rcParams['font.family'] = 'NanumGothic'
mpl.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

orgDf = pd.read_csv('../../datasets/LetalCarOfContractType.csv')
orgDf.head()

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,8.0,우리은행,정상,60.0,2363.0
3,66423450,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,계약확정,0,없음,5.0,농협은행,정상,60.0,2449.0
4,66423204,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,해약확정,12,있음,8.0,농협은행,해약,51.0,1942.0


### 범주형 / 연속형 : 범주형 따른 관계 확인


### 예비 가설 
- 가설 : 채널 중 서비스 방문과 홈쇼핑 항목 따른 월 렌탈 비용 평균이 차이가 있는가 ?
    + 귀무 : 평균에 차이가 없다
    + 대립 : 평균에 차이가 있따

In [33]:
### 정규분포 여부 확인 : channel ; 서비스 방문 & 홈쇼핑/방송


conditions = 'channel in ("서비스 방문", "홈쇼핑/방송")' # 같은 컬럼이기에 in을 쓸 수 있다.

channelTwo = orgDf.query(conditions)
len(channelTwo)


28163

In [43]:
# 범주형은 카이제곱 말고는 방법이 없다, 또한 정규분포는 > 연속형 타입만 가능함

# 연속형 타입 > anderson

result = stats.anderson(channelTwo['amount']) # 정규분포 
result

AndersonResult(statistic=1292.5765966678482, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=94187.21940134218, scale=10213.602310959697)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')

In [35]:
orgDf['channel'].value_counts().iloc[:2]

channel
서비스 방문    15186
홈쇼핑/방송    12977
Name: count, dtype: int64

In [36]:
condChnFrst = 'channel == "서비스 방문"'
# orgDf.loc[(orgDf['channel'] == '서비스 방문')]

chnFirstDf = orgDf.query(condChnFrst)
len(chnFirstDf) # 갯수가 동일한지 확인을 통해 잘 나오는지 확인

15186

In [37]:
condChnScnd = 'channel == "홈쇼핑/방송"'
chnScndDf = orgDf.query(condChnScnd)
len(chnScndDf) # 갯수가 동일한지 확인을 통해 잘 나오는지 확인

12977

### 연속형이 비정규 분포이며, 범주형 항목이 2개일 때
- Wilcoxon Test

In [38]:
import scipy.stats as stats 

In [39]:
stats.ranksums(chnFirstDf['amount'], chnScndDf['amount'])

RanksumsResult(statistic=74.731078009603, pvalue=0.0)

### 연속형이 비정규 분포이며, 범주형 항목이 3개일 때
- 또 다른 Test

In [40]:
orgDf['channel'].value_counts().iloc[:3]

channel
서비스 방문    15186
홈쇼핑/방송    12977
렌탈총판       5544
Name: count, dtype: int64

In [41]:
condChnThrd = 'channel == "렌탈총판"'
chnThrdDf = orgDf.query(condChnThrd)
len(chnThrdDf) # 갯수가 동일한지 확인을 통해 잘 나오는지 확인

5544

In [42]:
test_list = [chnFirstDf['amount'], chnScndDf['amount'], chnThrdDf['amount']]
stats.kruskal(*test_list)

KruskalResult(statistic=8232.430933915304, pvalue=0.0)